In [1]:
# --- 匯入套件 ---
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
# --- Step 1: 讀取 CSV ---
csv_path = "./dance_csv/Ballet_1.csv"  # 檔案名稱
df = pd.read_csv(csv_path)

In [3]:
# 移除 frame 欄，只取骨架點
pose_cols = [c for c in df.columns if c != "frame"]

In [4]:
# 把每個 ('x','y','z') 的字串轉成數字
def parse_point(s):
    # 去除括號與引號
    s = s.strip("()")
    parts = [float(p.strip(" '")) for p in s.split(",")]
    return parts

In [5]:
# 每幀 → 33 個點 × 3 維 = 99 維向量
poses = []
for i, row in df.iterrows():
    pose = []
    for c in pose_cols:
        pose += parse_point(row[c])
    poses.append(pose)

poses = np.array(poses)
print("骨架資料 shape:", poses.shape)  # (N_frames, 99)

骨架資料 shape: (241, 99)


In [6]:
# --- Step 2: 正規化 ---
scaler = StandardScaler()
poses = scaler.fit_transform(poses)

In [7]:
# --- Step 3: 建立資料集 ---
class PoseDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data, dtype=torch.float32)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]

dataset = PoseDataset(poses)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [8]:
# --- Step 4: 定義 VQ-VAE ---
class VectorQuantizer(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.uniform_(-1/num_embeddings, 1/num_embeddings)

    def forward(self, x):
        # x: (batch, dim)
        distances = (
            torch.sum(x**2, dim=1, keepdim=True)
            + torch.sum(self.embedding.weight**2, dim=1)
            - 2 * torch.matmul(x, self.embedding.weight.t())
        )
        encoding_indices = torch.argmin(distances, dim=1)
        quantized = self.embedding(encoding_indices)
        return quantized, encoding_indices

class VQVAE(nn.Module):
    def __init__(self, input_dim=99, hidden_dim=64, latent_dim=32, num_embeddings=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )
        self.vq = VectorQuantizer(num_embeddings, latent_dim)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        z_q, indices = self.vq(z)
        x_recon = self.decoder(z_q)
        return x_recon, indices, z, z_q

In [9]:
# --- Step 5: 訓練 ---
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VQVAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(20):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        x_recon, indices, z, z_q = model(batch)
        recon_loss = loss_fn(x_recon, batch)
        vq_loss = torch.mean((z_q.detach() - z)**2) + 0.25 * torch.mean((z_q - z.detach())**2)
        loss = recon_loss + vq_loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/20, Loss: {total_loss/len(dataloader):.4f}")

Epoch 1/20, Loss: 1.0644
Epoch 2/20, Loss: 1.0339
Epoch 3/20, Loss: 1.0090
Epoch 4/20, Loss: 1.0163
Epoch 5/20, Loss: 1.0026
Epoch 6/20, Loss: 1.0136
Epoch 7/20, Loss: 1.0017
Epoch 8/20, Loss: 1.0006
Epoch 9/20, Loss: 1.0215
Epoch 10/20, Loss: 1.0040
Epoch 11/20, Loss: 0.9951
Epoch 12/20, Loss: 1.0036
Epoch 13/20, Loss: 1.0058
Epoch 14/20, Loss: 0.9945
Epoch 15/20, Loss: 0.9955
Epoch 16/20, Loss: 0.9953
Epoch 17/20, Loss: 0.9938
Epoch 18/20, Loss: 0.9824
Epoch 19/20, Loss: 0.9814
Epoch 20/20, Loss: 0.9664


In [10]:
# --- Step 6: 轉換成符號序列 ---
model.eval()
with torch.no_grad():
    all_indices = []
    for batch in dataloader:
        batch = batch.to(device)
        _, indices, _, _ = model(batch)
        all_indices.extend(indices.cpu().numpy())

In [11]:
# --- Step 7: 生成符號對應 ---
symbols = [chr(65 + (i % 26)) for i in all_indices]  # A-Z 符號循環
sequence = "".join(symbols)
print("符號序列預覽：", sequence[:200])

符號序列預覽： QWNMQTAJBRVTKBNBJNYGOEGGJUJEYBKHJPFFESJCBETBKUGUPEJDIJJRFDMMFJJKJHJGJJJIJTEBBTTUEGBITDKNLGUBJFVWFJYWFOENTQJJCEKTTQJDLOGYOLXJUROTBWTJFNQENTWLJNRNGJHFOIMWULWODPBJSMOOTLUEACQEENTJPLJMYQFOGJPAIJPJBQNLJOSL
